In [10]:
! pip install xlsxwriter


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pdfplumber
import re
import pandas as pd
from collections import defaultdict
import glob
import os
import statistics

# ===================== PARÁMETROS AJUSTABLES =====================
FOLDER_PATH = "data/"
OUTPUT_FOLDER = "outputs/"
RIGHT_CUT_FRAC = 0.74          # corte para ignorar columna derecha (x1 <= width * RIGHT_CUT_FRAC)
LEVEL_LEFT_MARGIN_FRAC = 0.08  # para detectar los dígitos grandes de nivel a la izquierda
HEIGHT_MIN_FRAC = 0.88         # descarta "words" cuyo alto < (mediana_alto * este_factor)
MIN_HEIGHT_PX = 7.5            # umbral de seguridad: si la mediana falla, usa este mínimo absoluto
# ================================================================

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# PDFs a procesar
pdf_files = glob.glob(os.path.join(FOLDER_PATH, "*.pdf"))

# Regex de códigos: 4 letras + 3 dígitos
CODE_RE = re.compile(r"\b[A-Z]{4}\d{3}\b")

# Mapa de símbolos a espacio para normalizar (flechas, guiones, bullets)
ARROW_CHARS = [
    "→","⇒","➔","⟶","⟼","⟿","↦","↠","➤","▶","❯","›","»",
    "-", "—", "–", "·", "•", "‣", "∙", "·"
]
ARROW_TABLE = str.maketrans({ch: " " for ch in ARROW_CHARS})

def normalize_token(txt: str) -> str:
    # Limpia flechas y símbolos, compacta espacios
    txt = (txt or "").translate(ARROW_TABLE)
    txt = re.sub(r"\s+", " ", txt.strip())
    return txt

all_rows = []

for pdf_path in pdf_files:
    file_name = os.path.splitext(os.path.basename(pdf_path))[0]
    levels_codes = defaultdict(list)
    carrera_name = "DESCONOCIDA"

    with pdfplumber.open(pdf_path) as pdf:
        # ------ Carrera ------
        first_text = pdf.pages[0].extract_text() or ""
        m = re.search(r"Carrera:\s*(.+)", first_text, flags=re.IGNORECASE)
        if m:
            carrera_name = m.group(1).strip()

        for page in pdf.pages:
            width, height = page.width, page.height

            # Extraemos palabras con coordenadas
            words = page.extract_words(
                x_tolerance=2, y_tolerance=2,
                keep_blank_chars=False, use_text_flow=True
            )

            # Ignorar columna derecha
            right_cut = width * RIGHT_CUT_FRAC
            left_words = [w for w in words if w["x1"] <= right_cut]

            # ------------------ FILTRO POR TAMAÑO ------------------
            # Calculamos la mediana del alto de los words (proxy de font-size)
            heights = [(w["bottom"] - w["top"]) for w in left_words if w["bottom"] > w["top"]]
            if heights:
                med_h = statistics.median(heights)
                height_threshold = max(med_h * HEIGHT_MIN_FRAC, MIN_HEIGHT_PX)
            else:
                height_threshold = MIN_HEIGHT_PX

            # Dejamos solo words "grandes" (evita códigos chiquitos de flechas/prerrequisitos)
            left_words_big = [w for w in left_words if (w["bottom"] - w["top"]) >= height_threshold]

            # ------------------ DETECCIÓN DE NIVELES ------------------
            level_markers = [
                {"text": w["text"], "y": (w["top"] + w["bottom"]) / 2}
                for w in left_words_big
                if w["text"] in list("123456789") and w["x0"] < width * LEVEL_LEFT_MARGIN_FRAC
            ]
            level_markers.sort(key=lambda d: d["y"])

            # Deduplicar por proximidad vertical
            dedup_levels = []
            for lm in level_markers:
                if not dedup_levels or abs(lm["y"] - dedup_levels[-1]["y"]) > 8:
                    dedup_levels.append(lm)

            # Rango Y por nivel
            ranges = []
            for i, lm in enumerate(dedup_levels):
                y_top = lm["y"] - 24
                y_bottom = (dedup_levels[i+1]["y"] - 24) if (i+1 < len(dedup_levels)) else height
                try:
                    lvl = int(lm["text"])
                    ranges.append((lvl, y_top, y_bottom))
                except:
                    pass

            # ------------------ EXTRACCIÓN DE CÓDIGOS ------------------
            # Normalizamos texto (para no perder códigos pegados a flechas/símbolos)
            for w in left_words_big:
                cleaned = normalize_token(w["text"]).upper()
                # Buscar TODAS las coincidencias (si vienen pegadas)
                for tok in re.findall(r"[A-Z]{4}\d{3}", cleaned):
                    if CODE_RE.fullmatch(tok):
                        y_center = (w["top"] + w["bottom"]) / 2
                        # Asignar por nivel según rango vertical
                        for lvl, y0, y1 in ranges:
                            if y0 <= y_center < y1:
                                levels_codes[lvl].append(tok)
                                break

    # Limpiar duplicados por nivel manteniendo orden
    for lvl in levels_codes:
        seen = set()
        ordered = []
        for c in levels_codes[lvl]:
            if c not in seen:
                ordered.append(c)
                seen.add(c)
        levels_codes[lvl] = ordered

    # Armar filas
    rows = []
    for lvl in sorted(levels_codes.keys()):
        for code in levels_codes[lvl]:
            rows.append({
                "archivo": file_name,
                "carrera": carrera_name,
                "nivel": lvl,
                "codigo": code
            })

    # Guardar por archivo
    df = pd.DataFrame(rows).sort_values(["nivel", "codigo"]).reset_index(drop=True)
    out_csv = os.path.join(OUTPUT_FOLDER, f"{file_name}_codigos_por_nivel.csv")
    df.to_csv(out_csv, index=False)
    print(f"Procesado: {pdf_path} -> {out_csv}")

    all_rows.extend(rows)

# Consolidado
df_all = pd.DataFrame(all_rows).sort_values(["archivo", "nivel", "codigo"]).reset_index(drop=True)
df_all.to_csv(os.path.join(OUTPUT_FOLDER, "todas_mallas_codigos.csv"), index=False)
print("✅ Listo. Consolidado en outputs/todas_mallas_codigos.csv")


Procesado: data\malla_administracion_empresas.pdf -> outputs/malla_administracion_empresas_codigos_por_nivel.csv
Procesado: data\malla_agroindustria.pdf -> outputs/malla_agroindustria_codigos_por_nivel.csv
Procesado: data\malla_ambiental.pdf -> outputs/malla_ambiental_codigos_por_nivel.csv
Procesado: data\malla_ciencia_datos_IA.pdf -> outputs/malla_ciencia_datos_IA_codigos_por_nivel.csv
Procesado: data\malla_civil.pdf -> outputs/malla_civil_codigos_por_nivel.csv
Procesado: data\malla_computacion.pdf -> outputs/malla_computacion_codigos_por_nivel.csv
Procesado: data\malla_economia.pdf -> outputs/malla_economia_codigos_por_nivel.csv


In [ ]:
import pandas as pd

# -------- EXCEPCIONES: estos códigos se ignoran al evaluar conflictos --------
EXCEPTION_CODES = {
    # (anteriores)
    "TITD101","TITD201","MATD113","MATD123","FISD134","MATD213","MATD223","MATD115",
    "MATD124","MATD133","MATD141","MATD224","MATD234","MATD314","FISD113","MATD143",
    "MATD153","CSHD111","CSHD211","AMBD261","CSHD162","CSHD262","CSHD362","ADMD511","ADMD611","ADMD711","ADMD163",
    "ADMD421","ICOD111","ICOD151","ICOD142","ICOD173","ICOD273","DEPD110","DEPD120",
    "ADMD700","ADMD800","AMBD900","CSHD600","CSHD311","CSHD321","CSHD331","CSHD341",
    "CSHD351","CSHD361","CSHD371","CSHD381","CSHD391","CSHD3A1","CSHD3B1","CSHD411",
    "CSHD421","CSHD431","CSHD441","CSHD451","CSHD510","CSHD520"
}

def is_exception(code: str) -> bool:
    """
    Regla centralizada de excepciones:
    - Códigos listados en EXCEPTION_CODES
    - Todo código que empiece por 'TITD'
    """
    if pd.isna(code):
        return True
    c = str(code).strip().upper()
    return (c in EXCEPTION_CODES) or c.startswith("TITD")

# ------------------ Cargar datos ------------------
# Base con asignaciones de profesores (tiene columnas: 'Nombre', 'SII - Código Materia', 'SII - Paralelo', etc.)
profesores_df = pd.read_excel("data/asignaturas.xlsx")

# Limpieza mínima
profesores_df["Nombre"] = profesores_df["Nombre"].astype(str).str.strip()

# Renombramos la columna de código para facilidad
profesores_df = profesores_df.rename(columns={"SII - Código Materia": "codigo"})
# (Opcional normalización de códigos)
# profesores_df["codigo"] = profesores_df["codigo"].astype(str).str.strip().str.upper()

# Cargar consolidado de mallas (tiene columnas: 'codigo', 'nivel', 'carrera')
mallas_df = pd.read_csv("outputs/todas_mallas_codigos.csv")
# (Opcional normalización de códigos)
# mallas_df["codigo"] = mallas_df["codigo"].astype(str).str.strip().str.upper()

# ------------------ Unir mallas a profesores ------------------
merged = profesores_df.merge(
    mallas_df[["codigo", "nivel", "carrera"]],
    on="codigo",
    how="left"
)

# ------------------ Mapa de múltiples paralelos ------------------
# Para cada código, contamos cuántos paralelos distintos existen en TODA la matriz.
# Si tiene >1 paralelo => True (múltiples paralelos).
multi_parallel_map = (
    profesores_df
    .dropna(subset=["codigo"])
    .groupby("codigo")["SII - Paralelo"]
    .nunique(dropna=True)
    .gt(1)  #
    .to_dict()
)

# ------------------ Construir observaciones ------------------
observaciones = {}

for nombre, grupo in merged.groupby("Nombre", dropna=False):
    problemas = []
    # Solo filas donde conocemos nivel y carrera
    grupo_ok = grupo.dropna(subset=["nivel", "carrera"], how="any")

    for (nivel, carrera), sub in grupo_ok.groupby(["nivel", "carrera"]):
        # Filtramos códigos no exceptuados
        cods_filtrados = [
            c for c in sub["codigo"]
            if pd.notna(c) and not is_exception(c)
        ]
        cods_unicos = sorted(set(cods_filtrados))

        # Hay conflicto si existen 2+ códigos distintos en el mismo nivel/carrera
        if len(cods_unicos) > 1:
            # Regla: solo quitamos la observación si TODAS estas asignaturas tienen >1 paralelo
            todas_tienen_multiples = all(multi_parallel_map.get(c, False) for c in cods_unicos)

            if not todas_tienen_multiples:
                # Al menos una tiene 1 solo paralelo -> se mantiene observación con TODOS los códigos en conflicto
                problemas.append(
                    f"Malla {carrera} – Nivel {int(nivel)}: {', '.join(cods_unicos)}"
                )
            # Si todas tienen >1 paralelo, NO agregamos observación (se elimina)

    observaciones[nombre] = ("• " + "\n• ".join(problemas)) if problemas else ""

# Agregar columna Observaciones al dataframe original
profesores_df["Observaciones"] = profesores_df["Nombre"].map(observaciones).fillna("")

# Dejar la observación SOLO en la primera fila de cada profesor
mask_dup = profesores_df.duplicated(subset=["Nombre"], keep="first")
profesores_df.loc[mask_dup, "Observaciones"] = ""

# ------------------ Exportar a Excel ------------------
output_file = "outputs/Reporte_AsignaturasPorNivel.xlsx"

with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    sheet_name = "Validación"
    profesores_df.to_excel(writer, index=False, sheet_name=sheet_name)

    workbook  = writer.book
    worksheet = writer.sheets[sheet_name]

    # Filtros por columna y congelar encabezado
    n_rows, n_cols = profesores_df.shape
    worksheet.autofilter(0, 0, n_rows, n_cols - 1)
    worksheet.freeze_panes(1, 0)

    # Formato wrap text para Observaciones
    wrap_fmt = workbook.add_format({"text_wrap": True, "valign": "top"})

    # Ajuste de ancho para la columna Observaciones
    obs_col_idx = list(profesores_df.columns).index("Observaciones")
    worksheet.set_column(obs_col_idx, obs_col_idx, 60, wrap_fmt)

print(f"✅ Archivo generado con observaciones en: {output_file}")


c:\Users\andra\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


✅ Archivo generado con observaciones en: outputs/Reporte_AsignaturasPorNivel.xlsx
